In [41]:
import pandas as pd
import config
import os
import sqlalchemy

In [42]:

_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.admin}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )
db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True})

In [43]:
directories = [x[0] for x in os.walk('/Users/schlinkertc/Flatiron/projects/classification/txt_files')][1:]
file_names = [x[2] for x in os.walk(directories[0])][0]

tag_paths = sorted([x+'/'+file_names[0] for x in directories],reverse=True)
pre_paths = sorted([x+'/'+file_names[1] for x in directories],reverse=True)
sub_paths = sorted([x+'/'+file_names[2] for x in directories],reverse=True)
num_paths = sorted([x+'/'+file_names[4] for x in directories],reverse=True)

#Dataframes for results from 2018-2019

tags = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='unicode_escape') for x in tag_paths[:7]
          ],axis=0
         )

presentations = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='utf-8') for x in pre_paths[:7]
          ],axis=0
         )

numbers = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='unicode_escape') for x in num_paths[:7]
          ],axis=0
         )

submissions = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='utf-8') for x in sub_paths[:7]
          ],axis=0
         )


In [44]:
submissions = submissions[submissions['form']=='10-K']

In [45]:
numbers = numbers[numbers['adsh'].isin(submissions['adsh'])]
presentations = presentations[presentations['adsh'].isin(submissions['adsh'])]

In [46]:
tags.drop(index=tags[tags['custom']==1].index,inplace=True)

In [70]:
ISnumbers = numbers.copy(deep=True)
ISnumbers = ISnumbers.join(presentations['stmt'],)
#ISnumbers = ISnumbers[ISnumbers['stmt']=='IS']

# ISdesired_values = [
#     'Revenues',
#     'GrossProfit',
#     'OperatingIncomeLoss',
#     'NetIncomeLoss',
#     'ProfitLoss',
#     'OperatingExpenses',    
# ]

ISdesired_values = ['NetIncomeLoss',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'OperatingIncomeLoss',
 'Revenues',
 'IncomeTaxExpenseBenefit',
 'CashAndCashEquivalentsAtCarryingValue',
 'StockholdersEquity',
 'GrossProfit',
 'NetCashProvidedByUsedInOperatingActivities',
 'ProfitLoss',
 'NetCashProvidedByUsedInFinancingActivities',
 'InterestExpense',
 'WeightedAverageNumberOfSharesOutstandingBasic',
 'Assets',
 'WeightedAverageNumberOfDilutedSharesOutstanding',
 'NetCashProvidedByUsedInInvestingActivities',
 'LiabilitiesAndStockholdersEquity',
 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
 'ShareBasedCompensation',
 'RetainedEarningsAccumulatedDeficit',
 'CashAndCashEquivalentsPeriodIncreaseDecrease',
 'ComprehensiveIncomeNetOfTax',
 'OperatingExpenses',
 'EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate']

ISnumbers = ISnumbers[ISnumbers['tag'].isin(ISdesired_values)]

In [71]:
ISnumbers.shape

(1030362, 10)

In [72]:
ISdf = ISnumbers.merge(submissions[['adsh','name','cik','sic','form','fy']],
               how='left',
               left_on='adsh',
               right_on='adsh')

In [73]:
ISdf.set_index(['cik'],inplace=True)
ISdf.drop(columns=['name','footnote','ddate','version','form','coreg','uom','stmt'],inplace=True)

In [74]:
ISdf = ISdf.loc[(ISdf['qtrs']==4) & (ISdf['fy'].astype('int')==2018)]

In [75]:
ISdf.drop_duplicates(inplace=True)

In [76]:
ISdf.drop(columns=['fy','qtrs'],inplace=True)

In [77]:
ISdf['manufacturing'] = ISdf['sic'].map(lambda x: 2000<x<4000)
ISdf.drop(columns='sic',inplace=True)

In [78]:
#ISdf.dropna(inplace=True)

In [79]:
piv = ISdf.reset_index().pivot_table(index=['cik','adsh'],columns='tag',values='value')

In [80]:
numbers['tag'].value_counts().sort_values(ascending=False).index[:25].tolist()

['NetIncomeLoss',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'OperatingIncomeLoss',
 'Revenues',
 'IncomeTaxExpenseBenefit',
 'CashAndCashEquivalentsAtCarryingValue',
 'StockholdersEquity',
 'GrossProfit',
 'NetCashProvidedByUsedInOperatingActivities',
 'ProfitLoss',
 'NetCashProvidedByUsedInFinancingActivities',
 'InterestExpense',
 'WeightedAverageNumberOfSharesOutstandingBasic',
 'Assets',
 'WeightedAverageNumberOfDilutedSharesOutstanding',
 'NetCashProvidedByUsedInInvestingActivities',
 'LiabilitiesAndStockholdersEquity',
 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
 'ShareBasedCompensation',
 'RetainedEarningsAccumulatedDeficit',
 'CashAndCashEquivalentsPeriodIncreaseDecrease',
 'ComprehensiveIncomeNetOfTax',
 'OperatingExpenses',
 'EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate']

In [81]:
len(ISdf.reset_index()['cik'].unique())

5592

In [84]:
piv.isna().sum(axis=0).sort_values()

tag
NetCashProvidedByUsedInOperatingActivities                                                      359
NetIncomeLoss                                                                                   379
NetCashProvidedByUsedInFinancingActivities                                                      385
NetCashProvidedByUsedInInvestingActivities                                                      711
IncomeTaxExpenseBenefit                                                                        1187
OperatingIncomeLoss                                                                            1309
ShareBasedCompensation                                                                         1591
EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate                            1629
InterestExpense                                                                                2102
ComprehensiveIncomeNetOfTax                                                                    2

In [83]:
piv.melt('')

,tag,CashAndCashEquivalentsPeriodIncreaseDecrease,ComprehensiveIncomeNetOfTax,EarningsPerShareBasic,EarningsPerShareDiluted,EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate,GrossProfit,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,InterestExpense,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,OperatingExpenses,OperatingIncomeLoss,ProfitLoss,Revenues,ShareBasedCompensation,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
cik,adsh,,,,,,,,,,,,,,,,,,,,
1750,0001047469-18-004978,NaN,4.273333e+07,1.156667,1.140000,0.321000,2.637000e+08,NaN,1.733333e+07,6.566667e+06,-1.350000e+07,-2.996667e+07,3.940000e+07,3.993333e+07,NaN,NaN,3.993333e+07,NaN,1.093333e+07,3.445000e+07,3.416667e+07
1800,0001047469-19-000624,NaN,1.166667e+09,0.850000,0.846667,0.280000,NaN,2.172333e+09,9.223333e+08,7.203333e+08,-1.508333e+09,-3.740667e+09,5.024333e+09,1.415000e+09,NaN,2.746667e+09,1.415000e+09,NaN,3.976667e+08,1.667333e+09,1.658333e+09
1961,0001264931-19-000040,NaN,NaN,-0.015000,-0.015000,NaN,NaN,NaN,NaN,-4.500000e+04,5.966910e+05,2.509572e+06,-1.229892e+06,-4.963265e+05,NaN,-2.958201e+06,NaN,NaN,NaN,NaN,5.113433e+07
2034,0001144204-18-051414,2.228467e+07,-8.792267e+07,-2.480000,-2.483333,0.315500,1.317133e+08,-7.386900e+07,1.612400e+07,1.454067e+07,6.071800e+07,-9.818433e+07,5.940133e+07,-8.999300e+07,NaN,-6.214333e+07,NaN,NaN,7.152333e+06,3.247633e+07,3.220300e+07
2098,0001026608-19-000011,-7.508640e+05,4.464522e+06,1.285000,1.195000,NaN,4.927413e+07,6.012260e+06,1.686846e+06,NaN,3.428758e+06,-8.805111e+06,4.643072e+06,4.325414e+06,NaN,7.669980e+06,NaN,1.339359e+08,7.767030e+05,3.633256e+06,3.363564e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756180,0001683168-19-001042,3.603500e+03,9.136555e+05,0.089450,0.089450,NaN,1.419624e+06,1.274171e+06,3.297615e+05,NaN,4.756115e+05,-3.420200e+04,-4.476640e+05,8.950870e+05,1.693445e+05,1.250280e+06,9.444095e+05,2.406394e+06,NaN,1.000322e+07,1.000322e+07
1757067,0001551163-19-000081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.642500e+04,NaN,-2.642500e+04,-2.656000e+04,2.656000e+04,NaN,-2.656000e+04,NaN,NaN,NaN,NaN
1757898,0001757898-19-000005,-9.402667e+06,2.050940e+08,2.766667,2.743333,0.171667,1.098129e+09,NaN,6.725633e+07,4.672133e+07,-3.060250e+08,-1.737693e+08,4.737410e+08,2.349770e+08,7.522773e+08,3.458513e+08,2.357763e+08,2.671641e+09,2.164867e+07,8.575833e+07,8.502600e+07
